## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime



In [2]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [3]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            test_att_values =[]
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        
                                        
                                        value_att = tx['value']
                        
                                        test_att_values.append(value_att)
                                        test_att = set(flatten_list(test_att_values))
                                        
                                        
                                        dictionary_2[child['actor']['agent']] = test_att
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [6]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            file_read = path.join(root, name)
            dir_name = (os.path.splitext(os.path.basename(root))[0])
            print(file_read)
            
            filename = (os.path.splitext(os.path.basename(file_read))[0])
            print(filename)
            with open(file_read,'r') as inf:
                query = json.load(inf)
                
                kcresult = submit_to_ars(query)
                
                sleep(600)
                
                result_status = retrieve_ars_results(kcresult)
                
        
                dict_workflows[filename] = kcresult
                            
                sleep(100)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.3_KCNMA1.json
A.3_KCNMA1
https://arax.ncats.io/?source=ARS&id=ad700e75-1d07-469d-a298-1f4ab9c8904a
Done
Done
ara-aragorn Done 117
Done
ara-arax Done 181
Done
ara-bte Done 31
Done
ara-unsecret Done 3
Done
kp-genetics Done 0
Done
kp-molecular Done 97
Error
ara-explanatory Error 0
Done
ara-improving Done 5
Error
kp-cam Error 0
Done
kp-textmining Done 14
Done
kp-openpredict Done 0
Done
kp-icees Done 0
Error
ara-robokop Error 0
Done
kp-chp Done 30
Done
kp-cohd ARS Error 0
Done
kp-icees-dili Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.2a_RHOBTB2_twohop.json
A.2a_RHOBTB2_twohop
https://arax.ncats.io/?source=ARS&id=f766302b-a20a-4717-b0cb-1e001a041756
Done
Error
ara-aragorn Error 0
Done
ara-arax Done 0
Done
ara-bte Done 0
Done
ara-unsecret Done 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Error
ara-explanatory Error 0
Done
ara-improving ARS Error 0
Error
kp-cam Error 0
Done


https://arax.ncats.io/?source=ARS&id=dec86ace-c166-4957-a8d0-0099e5f53f7e
Running
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.3c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin.json
B.3c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin
https://arax.ncats.io/?source=ARS&id=4f536cd0-c0c2-43a1-bc79-7b204e963b60
Running
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.2e_DILI-three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver.json
B.2e_DILI-three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver
https://arax.ncats.io/?source=ARS&id=3efe7f66-a0a8-43e3-b879-d5d494e7721e
Running
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.7_DILI_branched-four-hop-from-MONDO:0005359_DILI.json
B.7_DILI_branched-four-hop-from-MONDO:0005359_DILI
https://arax.ncats.io/?source=ARS&id=488352e3-7c10-407d-819c-4c075668bc2a
Running
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.1a_DILI-three-hop-from-MOND

ara-explanatory Error 0
Done
ara-improving Done 78
Error
kp-cam Error 0
Done
kp-textmining Done 0
Done
kp-openpredict Done 0
Done
kp-icees ARS Error 0
Error
ara-robokop Error 0
Done
kp-chp Done 0
Error
kp-cohd Error 0
Done
kp-icees-dili ARS Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowD/D.2_ssri-heart-disease.json
D.2_ssri-heart-disease
https://arax.ncats.io/?source=ARS&id=a8bd17dc-0a06-45a1-be14-e4a4b04aa8de
Running
Done
ara-aragorn Done 5199
Done
ara-arax Done 500
Error
ara-bte Error 0
Error
ara-unsecret Error 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Error
ara-explanatory Error 0
Done
ara-improving Done 32
Error
kp-cam Error 0
Running
kp-textmining Running 0
Done
kp-openpredict Done 0
Done
kp-icees ARS Error 0
Error
ara-robokop Error 0
Running
kp-chp Running 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowD/D.4_tryptophan-kynurenine.json
D.4_tryptophan-kynurenine
https://arax.ncats.io/?source=ARS&id=59ee7411-2f70-

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [7]:
dict_workflows

{'A.3_KCNMA1': 'ad700e75-1d07-469d-a298-1f4ab9c8904a',
 'A.2a_RHOBTB2_twohop': 'f766302b-a20a-4717-b0cb-1e001a041756',
 'A.8_EGFR_simple': '378badf3-37da-4e81-912c-9eddaadfad13',
 'A.Multiomics_BigGIM_DR_KP_RHOBTB2': '6e8f772d-8c34-4876-ae97-ce05bf0a8f1d',
 'A.0_RHOBTB2_direct': 'c0f3203d-0d14-4e95-a763-e1cb62add698',
 'A.2_RHOBTB2_twohop': 'fddd26aa-4db9-4d68-a6c2-e5e689c762e7',
 'A.Multiomics_BigGIM_DR_KP_EGFR': '69c6b1f9-2353-4ded-b4fb-28ca2fc6bee8',
 'A.2_RHOBTB2_twohop_constrained': '95522667-aac5-4342-87cd-69c5e8a83630',
 'A.9_EGFR_advanced': '5ff42504-c3d1-4736-a92f-61f1b9868ff9',
 'A.2a_expanded_RHOBTB2_twohop_constrained': 'd6734ddf-5221-4a89-a428-869dc655cf9f',
 'A.1_RHOBTB2': 'e9d65ab3-aaae-4bdf-afb0-6e669e5e716f',
 'A.2a_expanded_RHOBTB2_twohop': '235d1119-0f4f-4e64-85d2-dcb26e2bed64',
 'B.5_DILI-one-hop-from-genes': 'a38e9e75-c263-442e-b1c0-a51913047988',
 'B.0_Initial_three-hop_MESH:D056487_ChronicDILI': '15f8bb30-ea01-46f1-b924-158529d6c665',
 'B.4d_DILI-fourth-one-hop-f

In [8]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

A.3_KCNMA1 ad700e75-1d07-469d-a298-1f4ab9c8904a
Done
Done
ara-aragorn Done 117
Done
ara-arax Done 181
Done
ara-bte Done 31
Done
ara-unsecret Done 3
Done
kp-genetics Done 0
Done
kp-molecular Done 97
Error
ara-explanatory Error 0
Done
ara-improving Done 5
Error
kp-cam Error 0
Done
kp-textmining Done 14
Done
kp-openpredict Done 0
Done
kp-icees Done 0
Error
ara-robokop Error 0
Done
kp-chp Done 30
Done
kp-cohd ARS Error 0
Done
kp-icees-dili Done 0
A.2a_RHOBTB2_twohop f766302b-a20a-4717-b0cb-1e001a041756
Done
Error
ara-aragorn Error 0
Done
ara-arax Done 0
Done
ara-bte Done 0
Done
ara-unsecret Done 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Error
ara-explanatory Error 0
Done
ara-improving ARS Error 0
Error
kp-cam Error 0
Done
kp-textmining Done 0
Done
kp-openpredict Done 0
Done
kp-icees ARS Error 0
Error
ara-robokop Error 0
Unknown
kp-chp Unknown 0
Error
kp-cohd Error 0
Done
kp-icees-dili ARS Error 0
A.8_EGFR_simple 378badf3-37da-4e81-912c-9eddaadfad13
Done
Done
ara-aragorn Do

ara-unsecret Done 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Error
ara-explanatory Error 0
Done
ara-improving ARS Error 0
Error
kp-cam Error 0
Done
kp-textmining Done 0
Done
kp-openpredict Done 0
Done
kp-icees ARS Error 0
Error
ara-robokop Error 0
Error
kp-chp Error 0
Error
kp-cohd Error 0
Done
kp-icees-dili ARS Error 0
B.7_DILI_branched-four-hop-from-MONDO:0005359_DILI 488352e3-7c10-407d-819c-4c075668bc2a
Done
Error
ara-aragorn ARS Error 0
Error
ara-arax ARS Error 0
Error
ara-bte ARS Error 0
Done
ara-unsecret Done 0
Error
kp-genetics ARS Error 0
Error
kp-molecular ARS Error 0
Error
ara-explanatory ARS Error 0
Error
ara-improving ARS Error 0
Error
kp-cam ARS Error 0
Error
kp-textmining ARS Error 0
Error
kp-openpredict ARS Error 0
Error
kp-icees ARS Error 0
Error
ara-robokop ARS Error 0
Error
kp-chp ARS Error 0
Error
kp-cohd ARS Error 0
Error
kp-icees-dili ARS Error 0
B.1a_DILI-three-hop-from-MONDO:0005359_DILI e8e67d2f-67fc-4936-a63f-a349dc0a6bbb
Done
Error
ara-aragorn 

ara-explanatory Error 0
Done
ara-improving ARS Error 0
Error
kp-cam Error 0
Done
kp-textmining Done 0
Done
kp-openpredict Done 0
Done
kp-icees ARS Error 0
Error
ara-robokop Error 0
Error
kp-chp Error 0
Error
kp-cohd Error 0
Done
kp-icees-dili ARS Error 0
D.1_parkinsons-crohns 47deecbe-5797-403e-87d6-b1da10a63392
Done
Done
ara-aragorn Done 4318
Error
ara-arax Error 0
Error
ara-bte Error 0
Error
ara-unsecret Error 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Error
ara-explanatory Error 0
Done
ara-improving Done 22
Error
kp-cam Error 0
Done
kp-textmining Done 0
Done
kp-openpredict Done 0
Done
kp-icees ARS Error 0
Error
ara-robokop Error 0
Done
kp-chp Done 0
Error
kp-cohd Error 0
Done
kp-icees-dili ARS Error 0
D.3_ssri-heart-disease-one-hop 29904f18-172d-45d1-890d-ee8f6a88f671
Done
Done
ara-aragorn Done 218
Done
ara-arax Done 16
Error
ara-bte Error 0
Error
ara-unsecret Error 0
Done
kp-genetics Done 0
Done
kp-molecular Done 0
Error
ara-explanatory Error 0
Done
ara-improving Do

### Creating dataframe for workflows with PK

<br>

In [9]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)


for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    #count = 0
    for key, value in workflow_result_messages[k][0].items():
        #count= count+1
    
        
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    #print(count)
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.0_Initial_three-hop_MESH:D056487_ChronicDILI
B.1a_DILI-three-hop-from-MONDO:0005359_DILI
B.1b_DILI_three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis
B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI
B.1d_DILI_three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease
B.1e_DILI_three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver
B.2a_DILI-three-hop-from-MONDO:0005359_DILI
B.2b_DILI-three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis
B.2c_DILI-three-hop-from-MESH:D056487_ChronicDILI
B.2d_DILI-three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease
B.2e_DILI-three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver
B.3a_DILI-fourth-one-hop-from-CHEBI:41879_Dexamethasone
B.3b_DILI-four

In [10]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

### Creating second table with edge attribute source

<br>

In [11]:
final_dict2 = defaultdict(dict)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    count = 0
    
    for key, value in workflow_result_messages[k][1].items():
        final_dict2[k][key] = value

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.0_Initial_three-hop_MESH:D056487_ChronicDILI
B.1a_DILI-three-hop-from-MONDO:0005359_DILI
B.1b_DILI_three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis
B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI
B.1d_DILI_three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease
B.1e_DILI_three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver
B.2a_DILI-three-hop-from-MONDO:0005359_DILI
B.2b_DILI-three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis
B.2c_DILI-three-hop-from-MESH:D056487_ChronicDILI
B.2d_DILI-three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease
B.2e_DILI-three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver
B.3a_DILI-fourth-one-hop-from-CHEBI:41879_Dexamethasone
B.3b_DILI-four

In [12]:
final_dict2

defaultdict(dict,
            {'A.0_RHOBTB2_direct': {'ara-improving': {'infores:improving-agent',
               'infores:lincs',
               'infores:spoke'}},
             'A.1_RHOBTB2': {'ara-aragorn': {'infores:aragorn',
               'infores:aragorn-ranker-ara',
               'infores:automat-robokop',
               'infores:lincs',
               'infores:rtx-kg2',
               'infores:spoke'},
              'ara-arax': {'infores:arax', 'infores:rtx-kg2'},
              'ara-bte': {'infores:biothings-semmeddb-gene',
               'infores:semmeddb',
               'infores:translator-biothings-explorer'},
              'ara-improving': {'infores:improving-agent',
               'infores:lincs',
               'infores:spoke'}},
             'A.2_RHOBTB2_twohop': {'ara-arax': {'infores:arax',
               'infores:automat-hetio',
               'infores:biothings-semmeddb-gene',
               'infores:chembl',
               'infores:connections-hypothesis',
       

In [13]:
final_dictassemble = []
for k, vs in final_dict2.items():
    #print(k,vs)
    for kv, v in vs.items():
        for t in v:
            final_dictassemble.append([k,kv,t])

In [14]:
final_dictassemble

[['A.0_RHOBTB2_direct', 'ara-improving', 'infores:lincs'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:improving-agent'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:spoke'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:lincs'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:aragorn-ranker-ara'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:rtx-kg2'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:aragorn'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:spoke'],
 ['A.1_RHOBTB2', 'ara-aragorn', 'infores:automat-robokop'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:rtx-kg2'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:arax'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:semmeddb'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:biothings-semmeddb-gene'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:translator-biothings-explorer'],
 ['A.1_RHOBTB2', 'ara-improving', 'infores:lincs'],
 ['A.1_RHOBTB2', 'ara-improving', 'infores:improving-agent'],
 ['A.1_RHOBTB2', 'ara-improving', 'infores:spoke'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infor

In [15]:
column_names = ['Workflow', 'ARS-KPs', 'Values']
df2 = pd.DataFrame(final_dictassemble, columns=column_names)
df2 = df2.astype(str)

In [16]:
df2.Values = df2.Values.apply(lambda x: x[2:-2] if ('[' in x) else x)

In [17]:
df2test = df2.groupby(['Workflow','Values'])['ARS-KPs'].agg(list)

In [18]:
df2test = pd.DataFrame(df2test.unstack().T)

In [19]:
df2test

Workflow,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.4c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
Values,,,,,,,,,,,,,,,,,,,,,
CTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,NaN,NaN
DisGeNET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,NaN,NaN
DrugCentral Indication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,NaN,NaN
JensenLab Knowledge AmyCo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,NaN
JensenLab Knowledge GHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:string,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-improving]
infores:text-mining-provider-targeted,NaN,NaN,[ara-bte],[ara-bte],NaN,NaN,"[ara-arax, ara-bte, kp-textmining]","[ara-arax, ara-bte, kp-textmining]",[ara-bte],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:translator-biothings-explorer,NaN,[ara-bte],"[ara-arax, ara-bte]",[ara-bte],NaN,[ara-bte],"[ara-arax, ara-bte, kp-textmining]","[ara-arax, ara-bte, kp-textmining]",[ara-bte],"[ara-arax, ara-bte]",...,[ara-bte],NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN,NaN,NaN


In [20]:
#df2test.drop([''], axis=0, inplace=True)

In [21]:
#df2test.index = df2test.index.map(lambda x: x[2:-2] if ('[' in x) else x)

In [22]:
df2test

Workflow,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.4c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
Values,,,,,,,,,,,,,,,,,,,,,
CTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,NaN,NaN
DisGeNET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,NaN,NaN
DrugCentral Indication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,NaN,NaN
JensenLab Knowledge AmyCo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,NaN
JensenLab Knowledge GHR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:string,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-improving]
infores:text-mining-provider-targeted,NaN,NaN,[ara-bte],[ara-bte],NaN,NaN,"[ara-arax, ara-bte, kp-textmining]","[ara-arax, ara-bte, kp-textmining]",[ara-bte],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:translator-biothings-explorer,NaN,[ara-bte],"[ara-arax, ara-bte]",[ara-bte],NaN,[ara-bte],"[ara-arax, ara-bte, kp-textmining]","[ara-arax, ara-bte, kp-textmining]",[ara-bte],"[ara-arax, ara-bte]",...,[ara-bte],NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN,NaN,NaN


In [23]:
#df2.replace([], 'None', regex=True,inplace=True)

In [24]:
#df2test = df2test.mask(df2test.applymap(type).eq(list) & ~df2test.astype(bool))

In [25]:
df2test = df2test.rename_axis(None)

In [26]:
df2test.columns.name = None

In [27]:
df2test.fillna('', inplace=True)

In [28]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.4c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
CTD,,,,,,,,,,,...,,,,,,[ara-aragorn],[ara-aragorn],,,
DisGeNET,,,,,,,,,,,...,,,,,,[ara-aragorn],[ara-aragorn],,,
DrugCentral Indication,,,,,,,,,,,...,,,,,,[ara-aragorn],[ara-aragorn],,,
JensenLab Knowledge AmyCo,,,,,,,,,,,...,,,,,,[ara-aragorn],,,,
JensenLab Knowledge GHR,,,,,,,,,,,...,,,,,,[ara-aragorn],,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:string,,,,,,,,,,,...,,,,,,,,,,[ara-improving]
infores:text-mining-provider-targeted,,,[ara-bte],[ara-bte],,,"[ara-arax, ara-bte, kp-textmining]","[ara-arax, ara-bte, kp-textmining]",[ara-bte],,...,,,,,,,,,,
infores:translator-biothings-explorer,,[ara-bte],"[ara-arax, ara-bte]",[ara-bte],,[ara-bte],"[ara-arax, ara-bte, kp-textmining]","[ara-arax, ara-bte, kp-textmining]",[ara-bte],"[ara-arax, ara-bte]",...,[ara-bte],,,,[ara-bte],,,,,
infores:ubergraph,,,,,,,,,,,...,,,,,,[ara-aragorn],,,,


## Sorting The index to push infores ID's that are compliant to Column L and M in INFORES Catalog


<br>
<br>

In [33]:
infores_catalog = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/InfoRes Catalog - Translator InfoRes Catalog.csv", header=1)

In [34]:
infores_catalog = infores_catalog[['id', 'name','translator category','has contributor']]

In [40]:
infores_catalog = infores_catalog[:335]

In [41]:
dict_map = {}
for i in df2test.index.values:
    if i in infores_catalog['id'].values:
        indices = infores_catalog[infores_catalog['id']==i].index[0]
        if pd.notnull(infores_catalog.iloc[indices]['has contributor']):
            dict_map[i] = infores_catalog.iloc[indices]['translator category']
        else:
            dict_map[i] = 'External Source'
    else:
        dict_map[i] = 'Illegal value'

In [42]:
dict_map

{'CTD': 'Illegal value',
 'DisGeNET': 'Illegal value',
 'DrugCentral Indication': 'Illegal value',
 'JensenLab Knowledge AmyCo': 'Illegal value',
 'JensenLab Knowledge GHR': 'Illegal value',
 'JensenLab Knowledge UniProtKB-KW': 'Illegal value',
 'JensenLab Text Mining': 'Illegal value',
 'Monarch': 'Illegal value',
 'UniProt Disease': 'Illegal value',
 'eRAM': 'Illegal value',
 'icees:dili': 'Illegal value',
 'infores:aragorn': 'ARA',
 'infores:aragorn-ranker-ara': 'Illegal value',
 'infores:arax': 'ARA',
 'infores:automat-biolink': 'KP',
 'infores:automat-cord19': 'KP',
 'infores:automat-covidkop': 'KP',
 'infores:automat-ctd': 'KP',
 'infores:automat-drug-central': 'KP',
 'infores:automat-gtopdb': 'KP',
 'infores:automat-hetio': 'KP',
 'infores:automat-hmdb': 'KP',
 'infores:automat-human-goa': 'KP',
 'infores:automat-pharos': 'KP',
 'infores:automat-robokop': 'KP',
 'infores:automat-uberongraph': 'KP',
 'infores:bindingdb': 'External Source',
 'infores:biothings-dgidb': 'KP',
 'info

In [43]:
df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog']=df2test.index.map(dict_map)

In [44]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.4d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin,Translator_Category_Complaint_to_ColL&M_InforesCatalog
CTD,,,,,,,,,,,...,,,,,[ara-aragorn],[ara-aragorn],,,,Illegal value
DisGeNET,,,,,,,,,,,...,,,,,[ara-aragorn],[ara-aragorn],,,,Illegal value
DrugCentral Indication,,,,,,,,,,,...,,,,,[ara-aragorn],[ara-aragorn],,,,Illegal value
JensenLab Knowledge AmyCo,,,,,,,,,,,...,,,,,[ara-aragorn],,,,,Illegal value
JensenLab Knowledge GHR,,,,,,,,,,,...,,,,,[ara-aragorn],,,,,Illegal value
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:string,,,,,,,,,,,...,,,,,,,,,[ara-improving],External Source
infores:text-mining-provider-targeted,,,[ara-bte],[ara-bte],,,"[ara-arax, ara-bte, kp-textmining]","[ara-arax, ara-bte, kp-textmining]",[ara-bte],,...,,,,,,,,,,KP
infores:translator-biothings-explorer,,[ara-bte],"[ara-arax, ara-bte]",[ara-bte],,[ara-bte],"[ara-arax, ara-bte, kp-textmining]","[ara-arax, ara-bte, kp-textmining]",[ara-bte],"[ara-arax, ara-bte]",...,,,,[ara-bte],,,,,,ARA
infores:ubergraph,,,,,,,,,,,...,,,,,[ara-aragorn],,,,,Illegal value


In [45]:
df2test['sort']=pd.Categorical(df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog'], ["KP", "ARA",'External Source', 'Illegal Value'])
df2test =df2test.sort_values(['sort'])
df2test  = df2test.rename_axis(None)

In [46]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin,Translator_Category_Complaint_to_ColL&M_InforesCatalog,sort
infores:cam-kp,,,,,,,,,,,...,,[ara-aragorn],,,,,,,KP,KP
infores:mychem-info,,,,,,,[ara-arax],,,,...,,,,,,,,,KP,KP
infores:spoke,[ara-improving],"[ara-aragorn, ara-improving]",[ara-improving],[ara-improving],[ara-aragorn],,[ara-improving],[ara-improving],[ara-improving],,...,[ara-improving],,,"[ara-aragorn, ara-improving]","[ara-aragorn, ara-improving]",[ara-improving],[ara-improving],[ara-improving],KP,KP
infores:connections-hypothesis,,,[ara-arax],[ara-arax],,,[kp-chp],,,,...,[kp-chp],,[kp-chp],,,,,,KP,KP
infores:cohd,,,,,,,,,,,...,,,"[ara-aragorn, ara-arax, kp-cohd]",[ara-aragorn],,,,,KP,KP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
infores:faers,,,,,,,,,,,...,,,,[ara-aragorn],,[ara-aragorn],,,Illegal value,NaN
infores:hetio,,,"[ara-arax, ara-bte]",,,,,,,,...,[ara-aragorn],,,[ara-aragorn],[ara-aragorn],[ara-aragorn],,,Illegal value,NaN
infores:molepro-kp,,,,,[ara-arax],,"[ara-arax, kp-molecular]","[ara-arax, kp-molecular]",[kp-molecular],[kp-molecular],...,[kp-molecular],[kp-molecular],,,,,,,Illegal value,NaN
infores:ontological-hierarchy,,,,,,,,,,,...,,,,[ara-aragorn],,,,,Illegal value,NaN


In [47]:
cols = df2test.columns.tolist()
cols = [cols[-2]] + cols[:-2]
df2test = df2test[cols]

In [48]:
df2test['Query Type'] = 'Sync'

df2test = df2test[['Query Type']+ list(df2test.columns[:-1])]

<br>

## Converting the Pk's to hyperlink

<br>

In [49]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [50]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [51]:
df.sort_index(inplace=True)

In [52]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,...,B.7_DILI_branched-four-hop-from-MONDO:0005359_DILI,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,Error: 422,Results: 200,Error: 598,Error: 422,Error: 422,Results: 200,Error: 422,Results: 200,Results: 200,Error: 422,...,No Results: 422,Results: 200,Error: 422,Error: 422,Error: 422,Results: 200,Results: 200,Results: 200,Error: 598,Error: 598
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,...,No Results: 400,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Error: 598,Error: 598
ara-bte,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,...,No Results: 400,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,No Results: 200,No Results: 200
ara-explanatory,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,...,No Results: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,...,No Results: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-robokop,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,...,No Results: 422,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500
ara-unsecret,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,No Results: 200,No Results: 200
kp-cam,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,...,No Results: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502
kp-chp,No Results: 200,No Results: 200,No Results: 200,Error: 400,Unknown: 503,No Results: 200,Error: 400,Results: 200,Error: 502,Error: 400,...,No Results: 598,Results: 200,Error: 400,Error: 400,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
kp-cohd,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,No Results: 200,Error: 400,Error: 400,...,No Results: 400,Results: 200,Error: 400,Error: 400,Error: 400,Error: 400,Error: 400,Error: 500,Error: 400,Error: 400


<br>

### Highlight the cells In Excel

<br>

In [53]:
def highlight(v):
    if v.startswith('Results'):
        return 'background-color: %s' % 'green'
    elif v.startswith('Error'):
        return 'background-color: %s' % 'red'
    elif v.startswith('No Results'):
        return 'background-color: %s' % 'yellow'
    elif v.startswith('ARS Error'):
        return 'background-color: %s' % 'blue'
    elif v.startswith('Unknown'):
        return 'background-color: %s' % 'magneta'
    else:
        return

In [54]:
styled = df.style.applymap(highlight)

In [55]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,A.Multiomics_BigGIM_DR_KP_RHOBTB2,B.0_Initial_three-hop_MESH:D056487_ChronicDILI,B.1a_DILI-three-hop-from-MONDO:0005359_DILI,B.1b_DILI_three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis,B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI,B.1d_DILI_three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease,B.1e_DILI_three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver,B.2a_DILI-three-hop-from-MONDO:0005359_DILI,B.2b_DILI-three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis,B.2c_DILI-three-hop-from-MESH:D056487_ChronicDILI,B.2d_DILI-three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease,B.2e_DILI-three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver,B.3a_DILI-fourth-one-hop-from-CHEBI:41879_Dexamethasone,B.3b_DILI-fourth-one-hop-from-MESH:D000077185_Resveratrol,B.3c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin,B.3d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.4a_DILI-fourth-one-hop-from-CHEBI:41879_Dexamethasone,B.4b_DILI-fourth-one-hop-from-MESH:D000077185_Resveratrol,B.4c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,B.7_DILI_branched-four-hop-from-MONDO:0005359_DILI,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,Error: 422,Results: 200,Error: 598,Error: 422,Error: 422,Results: 200,Error: 422,Results: 200,Results: 200,Error: 422,Results: 200,Error: 422,Error: 422,Error: 598,Error: 598,Error: 422,Error: 500,Error: 422,Error: 422,Error: 598,Error: 422,Error: 598,Error: 598,Results: 200,Error: 422,Error: 422,Error: 422,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 422,Results: 200,Error: 422,Error: 422,Error: 422,Results: 200,Results: 200,Results: 200,Error: 598,Error: 598
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 400,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Error: 598,Error: 598
ara-bte,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,No Results: 200,Error: 502,Error: 598,Error: 598,Error: 598,No Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,No Results: 200,No Results: 200,Error: 598,Error: 598,Results: 200,Error: 502,No Results: 200,No Results: 200,No Results: 400,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,No Results: 200,No Results: 200
ara-explanatory,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,No Results: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 5

In [56]:
date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
wks_name = 'Workflow Progress Tracker_' + date

In [57]:
wks_name

'Workflow Progress Tracker_2021_09_23-07_47_01_AM'

In [58]:
styled.to_excel('/Users/priyash/Documents/GitHub/minihackathons/Notebooks/' + wks_name + '.xlsx')

#styled.to_excel(writer, sheet_name='progress_tracker')
#df2.to_excel(writer, sheet_name='edge_attribute')

/usr/local/lib/python3.9/site-packages/pandas/io/formats/excel.py:375: CSSWarning: Unhandled color format: 'magneta'
  warnings.warn(f"Unhandled color format: {repr(val)}", CSSWarning)


<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

In [59]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,A.Multiomics_BigGIM_DR_KP_RHOBTB2,B.0_Initial_three-hop_MESH:D056487_ChronicDILI,B.1a_DILI-three-hop-from-MONDO:0005359_DILI,B.1b_DILI_three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis,B.1c_DILI_three-hop-from-MESH:D056487_ChronicDILI,B.1d_DILI_three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease,B.1e_DILI_three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver,B.2a_DILI-three-hop-from-MONDO:0005359_DILI,B.2b_DILI-three-hop-from-SNOMEDCT:197358007_ToxicLiverDiseaseAcuteHepatitis,B.2c_DILI-three-hop-from-MESH:D056487_ChronicDILI,B.2d_DILI-three-hop-from-SNOMEDCT:197354009_ToxicLiverDisease,B.2e_DILI-three-hop-from-SNOMEDCT:427399008_DrugInducedDisorderLiver,B.3a_DILI-fourth-one-hop-from-CHEBI:41879_Dexamethasone,B.3b_DILI-fourth-one-hop-from-MESH:D000077185_Resveratrol,B.3c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin,B.3d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.4a_DILI-fourth-one-hop-from-CHEBI:41879_Dexamethasone,B.4b_DILI-fourth-one-hop-from-MESH:D000077185_Resveratrol,B.4c_DILI-fourth-one-hop-from-MESH:D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH:D003474_Curcumin,B.5_DILI-one-hop-from-genes,B.6_one-hop-gene-biological-process-or-activity_trapi,B.7_DILI_branched-four-hop-from-MONDO:0005359_DILI,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,Error: 422,Results: 200,Error: 598,Error: 422,Error: 422,Results: 200,Error: 422,Results: 200,Results: 200,Error: 422,Results: 200,Error: 422,Error: 422,Error: 598,Error: 598,Error: 422,Error: 500,Error: 422,Error: 422,Error: 598,Error: 422,Error: 598,Error: 598,Results: 200,Error: 422,Error: 422,Error: 422,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 422,Results: 200,Error: 422,Error: 422,Error: 422,Results: 200,Results: 200,Results: 200,Error: 598,Error: 598
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 400,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Error: 598,Error: 598
ara-bte,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,No Results: 200,Error: 502,Error: 598,Error: 598,Error: 598,No Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,No Results: 200,No Results: 200,Error: 598,Error: 598,Results: 200,Error: 502,No Results: 200,No Results: 200,No Results: 400,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,No Results: 200,No Results: 200
ara-explanatory,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,No Results: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 5

In [60]:
df['Query Type'] = 'Sync'

df = df[['Query Type']+ list(df.columns[:-1])]

In [61]:
#df.reset_index(inplace=True)

In [62]:
#df.rename({'index': 'ARA/KP'}, axis=1, inplace=True)

In [63]:
df.loc['pk'][1]

'=HYPERLINK("https://arax.ncats.io/?source=ARS&id=c0f3203d-0d14-4e95-a763-e1cb62add698", "https://arax.ncats.io/?source=ARS&id=c0f3203d-0d14-4e95-a763-e1cb62add698")'

<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

### Push Dataframe 1

<br>

In [64]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *

In [65]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [66]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Workflow Progress Tracker_2021_09_23-07_47_01_AM' id:1793056334>

### Push Dataframe 2

<br>

In [67]:
wks2 = 'edge_attribute_source_' + date

In [68]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df2test, spreadsheet_key, wks2, credentials=credentials, row_names=True)

<Worksheet 'edge_attribute_source_2021_09_23-07_47_01_AM' id:108447852>

In [69]:
wks2

'edge_attribute_source_2021_09_23-07_47_01_AM'

In [70]:
wks_name

'Workflow Progress Tracker_2021_09_23-07_47_01_AM'